https://medium.com/thecyphy/train-cnn-model-with-pytorch-21dafb918f48

In [1]:
import torch
print(torch.cuda.is_available(), torch.cuda.device_count())


True 1


In [2]:
import torch
print(torch.cuda.is_available(), torch.cuda.device_count())
import torchvision
from torchvision import transforms
from torchvision.datasets import ImageFolder
import glob
import shutil
import torch.nn as nn
import torch.nn.functional as F

True 1


In [3]:
device = torch.device("cuda")

Creat a subfolder with the correct label

In [4]:
orig_images = '/data/kanferg/Images/Pex_project/SIngle_cell_images_training_set/01242024/input_sc_mix_10000/'
files = glob.glob(orig_images + "*png")
trainPEX_dir = '/data/kanferg/Images/Pex_project/SIngle_cell_images_training_set/01242024/pytorch_test/seg_train/PEX'
trainWT_dir = '/data/kanferg/Images/Pex_project/SIngle_cell_images_training_set/01242024/pytorch_test/seg_train/WT'
testPEX_dir = '/data/kanferg/Images/Pex_project/SIngle_cell_images_training_set/01242024/pytorch_test/seg_test/PEX'
testWT_dir = '/data/kanferg/Images/Pex_project/SIngle_cell_images_training_set/01242024/pytorch_test/seg_test/WT'

In [5]:
filePEX = []
fileWT = []
for file in files:
    if 'WT' in os.path.basename(file):
        fileWT.append(file)
    else:
        filePEX.append(file)
fileWTCount = int(len(fileWT)*0.93)
filePEXCount = int(len(filePEX)*0.93)
print(fileWTCount),print(filePEXCount)

NameError: name 'os' is not defined

In [6]:
# for i in range(7500):
#     shutil.copy(fileWT[i], trainWT_dir)
#     shutil.copy(filePEX[i], trainPEX_dir)
#     print('.' * i, end = '\r')
# for i in range(7501,7773):
#     shutil.copy(fileWT[i], testWT_dir)
#     shutil.copy(filePEX[i], testPEX_dir)
#     print('.' * i, end = '\r')

In [4]:
data_dir = '/data/kanferg/Images/Pex_project/SIngle_cell_images_training_set/01242024/pytorch_test/seg_train/'
test_data_dir = '/data/kanferg/Images/Pex_project/SIngle_cell_images_training_set/01242024/pytorch_test/seg_test/'

In [5]:
dataset = ImageFolder(data_dir,transform = transforms.Compose([
    transforms.Resize((150,150)),transforms.ToTensor()
]))
test_dataset = ImageFolder(test_data_dir,transforms.Compose([
    transforms.Resize((150,150)),transforms.ToTensor()
]))


In [6]:
img, label = dataset[0]
print(img.shape,label)


torch.Size([3, 150, 150]) 0


In [7]:
print("Follwing classes are there : \n",dataset.classes)

Follwing classes are there : 
 ['PEX', 'WT']


In [8]:
from torch.utils.data.dataloader import DataLoader
from torch.utils.data import random_split

batch_size = 30
val_size = 3000
train_size = len(dataset) - val_size 

train_data,val_data = random_split(dataset,[train_size,val_size])
print(f"Length of Train Data : {len(train_data)}")
print(f"Length of Validation Data : {len(val_data)}")

#output
#Length of Train Data : 15000*0.8
#Length of Validation Data : 15000*0.2

#load the train and validation into batches.
# train_dl = DataLoader(train_data, batch_size, shuffle = True, num_workers = 4, pin_memory = True)
# val_dl = DataLoader(val_data, batch_size, num_workers = 4, pin_memory = True)
train_dl = DataLoader(train_data, batch_size, shuffle = True, pin_memory = True)
val_dl = DataLoader(val_data, batch_size, pin_memory = True)

Length of Train Data : 12000
Length of Validation Data : 3000


In [12]:
for i, data in enumerate(train_dl, 0):
    # get the inputs; data is a list of [inputs, labels]
    inputs, labels = data
    if i  == 0:
        break

In [13]:
inputs.shape

torch.Size([30, 3, 150, 150])

In [13]:
labels.view(-1, 1) .shape

torch.Size([30, 1])

using this from here: https://medium.com/@golnaz.hosseini/beginner-tutorial-image-classification-using-pytorch-63f30dcc071c

In [14]:
inputs.dtype

torch.float32

In [15]:
for i, data in enumerate(train_dl, 0):
    # get the inputs; data is a list of [inputs, labels]
    inputs, labels = data
    if i  == 0:
        break
labels = labels.view(-1, 1)
input_tensor = torch.randn(30, 3, 150, 150)
b1 = nn.Sequential(nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3),
                            nn.BatchNorm2d(16),
                            nn.ReLU(),
                            nn.MaxPool2d(2,2))
b1(input_tensor).shape
b2 = nn.Sequential(nn.Conv2d(in_channels=16, out_channels=64, kernel_size=3),
                            nn.BatchNorm2d(64),
                            nn.ReLU(),
                            nn.MaxPool2d(2,2)
                            )
b3 = nn.Sequential(nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3),
                            nn.BatchNorm2d(128),
                            nn.ReLU(),
                            nn.MaxPool2d(2,2)
                            )
b4 = nn.Sequential(nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3),
                            nn.BatchNorm2d(128),
                            nn.ReLU(),
                            nn.MaxPool2d(2,2)
                            )
b5 = nn.Sequential(nn.Flatten(),
                                        nn.Linear(7*7*128, 512),
                                        nn.ReLU(),
                                        nn.Dropout(0.3),
                                        nn.Linear(512, 512), 
                                        nn.ReLU(),
                                        nn.Dropout(0.3),
                                        nn.Linear(512, 1),
                                        nn.Softmax()
                                        )
b5(b4(b3(b2(b1(input_tensor))))).shape
criterion = nn.BCEWithLogitsLoss()
loss = criterion(b5(b4(b3(b2(b1(input_tensor))))), labels.float())
loss.backward()

/usr/local/Anaconda/envs/py3.9/lib/python3.9/site-packages/torch/nn/modules/container.py:141: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


In [9]:
from IPython.core.debugger import set_trace
class CustomCNN(nn.Module):
    def __init__(self):
        super(CustomCNN, self).__init__()
        
        self.layer1 = self.ConvModule(in_features=3, out_features=16) # 0, 16, 74, 74      
        self.layer2 = self.ConvModule(in_features=16, out_features=64) # 64, 36, 36
        self.layer3 = self.ConvModule(in_features=64, out_features=128)   #128, 17, 17
        self.layer4 = self.ConvModule(in_features=128, out_features=128)   #128,7,7

        self.classifier = nn.Sequential(nn.Flatten(),
                                        nn.Linear(7*7*128, 512), #512
                                        nn.ReLU(),
                                        nn.Dropout(0.3),
                                        nn.Linear(512, 512), #512
                                        nn.ReLU(),
                                        nn.Dropout(0.3),
                                        nn.Linear(512, 1),
                                        nn.Softmax())
    
    def forward(self, x):
        #set_trace()
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.classifier(x)
        return x
    
    def ConvModule(self, in_features, out_features):
        return nn.Sequential(nn.Conv2d(in_channels=in_features, out_channels=out_features, kernel_size=3),
                            #nn.BatchNorm2d(out_features),
                            nn.ReLU(),
                            nn.MaxPool2d(2,2)
                            )

    

In [10]:
model = CustomCNN().to(device)
#model = CustomCNN()
#model(inputs)

In [11]:
model = CustomCNN().to(device)

In [12]:
from IPython.core.debugger import set_trace

In [18]:
# dummy_input = torch.randn(1,3, 150, 150)  # Create the dummy input
# dummy_input = dummy_input.to('cpu')# Move the dummy input to the same device
# with torch.no_grad():  # We don't need to compute gradients for this
#     def print_shapes(x, module):
#         print(f"Layer: {module.__class__.__name__}, Output shape: {x.shape}")
#     hooks = []
#     for layer in model.children():
#         hook = layer.register_forward_hook(print_shapes)
#         hooks.append(hook)
#     # Forward pass
#     model(dummy_input)  # This will print out the shapes
#     # Unregister hooks
#     for hook in hooks:
#         hook.remove()

In [14]:
learning_rate = 0.001
num_epochs = 30
criterion = nn.BCEWithLogitsLoss()
optimizer= torch.optim.SGD(model.parameters(), lr=0.001)

In [ ]:
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(train_dl, 0):
        # get the inputs; data is a list of [inputs, labels]
        #inputs, labels = data
        set_trace()
        inputs, labels = data[0].to(device), data[1].to(device)  
        
        # zero the parameter gradients
        
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        #set_trace()
        labels = labels.view(-1, 1)
        loss = criterion(outputs, labels.float())
        #print(loss)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

In [ ]:
train_dl

In [ ]:
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(train_dl, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

print('Finished Training')

In [14]:
def train_model(model, dataloaders, criterion, optimizer, num_epochs):
    since = time.time()

    test_acc_history = []
    test_loss_history = []
    train_acc_history = []
    train_loss_history = []

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch+1, num_epochs))

        for phase in ['train', 'test']:
            if phase == 'train':
                model.train()  # training mode
            else:
                model.eval()   # evaluate mode

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)

                    _, preds = torch.max(outputs, 1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print('{} Loss: {:.4f}, Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            if phase == 'test':
                test_acc_history.append(epoch_acc.cpu().numpy())
                test_loss_history.append(epoch_loss)
            if phase == 'train':
                train_acc_history.append(epoch_acc.cpu().numpy())
                train_loss_history.append(epoch_loss)

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best test Acc: {:4f}'.format(best_acc))

    history_dict = {'train_loss':train_loss_history, 'train_accuracy':train_acc_history,
                    'test_loss':test_loss_history, 'test_accuracy':test_acc_history}
    return model, history_dict

In [16]:
import time
import copy
train_model(model, dataloaders, criterion, optimizer, num_epochs)

Epoch 1/30


RuntimeError: cuDNN error: CUDNN_STATUS_ARCH_MISMATCH

In [15]:
import torch.nn as nn
import torch.nn.functional as F

class ImageClassificationBase(nn.Module):
    
    def training_step(self, batch):
        images, labels = batch 
        out = self(images)                  # Generate predictions
        loss = F.cross_entropy(out, labels) # Calculate loss
        return loss
    
    def validation_step(self, batch):
        images, labels = batch 
        out = self(images)                    # Generate predictions
        loss = F.cross_entropy(out, labels)   # Calculate loss
        acc = accuracy(out, labels)           # Calculate accuracy
        return {'val_loss': loss.detach(), 'val_acc': acc}
        
    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
        batch_accs = [x['val_acc'] for x in outputs]
        epoch_acc = torch.stack(batch_accs).mean()      # Combine accuracies
        return {'val_loss': epoch_loss.item(), 'val_acc': epoch_acc.item()}
    
    def epoch_end(self, epoch, result):
        print("Epoch [{}], train_loss: {:.4f}, val_loss: {:.4f}, val_acc: {:.4f}".format(
            epoch, result['train_loss'], result['val_loss'], result['val_acc']))

In [30]:
# Forward a dummy input through the network and print layer outputs
dummy_input = torch.randn(3, 150, 150)
dummy_input = dummy_input.to(device) # Batch size of 1, 3 channels, 150x150 image
with torch.no_grad():  # We don't need to compute gradients for this
    def print_shapes(x, module):
        print(f"Layer: {module.__class__.__name__}, Output shape: {x.shape}")
    hooks = []
    for layer in model.children():
        hook = layer.register_forward_hook(print_shapes)
        hooks.append(hook)
    # Forward pass
    model(dummy_input)  # This will print out the shapes
    # Unregister hooks
    for hook in hooks:
        hook.remove()

RuntimeError: cuDNN error: CUDNN_STATUS_ARCH_MISMATCH

In [86]:
from torch.utils.data.dataloader import DataLoader
from torch.utils.data import random_split

batch_size = 30
val_size = 3000
train_size = len(dataset) - val_size 

train_data,val_data = random_split(dataset,[train_size,val_size])
print(f"Length of Train Data : {len(train_data)}")
print(f"Length of Validation Data : {len(val_data)}")

#output
#Length of Train Data : 15000*0.8
#Length of Validation Data : 15000*0.2

#load the train and validation into batches.
train_dl = DataLoader(train_data, batch_size, shuffle = True, num_workers = 4, pin_memory = True)
val_dl = DataLoader(val_data, batch_size*2, num_workers = 4, pin_memory = True)

Length of Train Data : 12000
Length of Validation Data : 3000


In [ ]:
size_after_convs = 150
for _ in range(4):  # 4 sets of Conv2D + MaxPooling2D
    size_after_convs = (size_after_convs - 2) // 2  # Kernel size 3, stride 1 for conv and 2 for pool
flat_size = size_after_convs * size_after_convs * 128  # The last number of channels after convolutions
flat_size

In [90]:
class NaturalSceneClassification(ImageClassificationBase):
    def __init__(self):
        super().__init__()
        self.network = nn.Sequential(
            
            nn.Conv2d(3, 16, kernel_size = (3,3), padding = 'valid'),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2, 2)), # Output size: [16, 74, 74]
        
            nn.Conv2d(16, 64, kernel_size = (3,3), stride = 1, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2, 2)), # Output size: [64, 36, 36]
            
            nn.Conv2d(64, 128, kernel_size = (3,3), stride = 1, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2, 2)), # Output size: [128, 17, 17]
            
            nn.Conv2d(128, 128, kernel_size = (3,3), stride = 1, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2, 2)), # Output size: [128, 7, 7]
            
            nn.Flatten(),
            nn.Linear(128 * 7 * 7, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            
            nn.Linear(512,512),
            nn.ReLU(),
            nn.Dropout(0.3),
            
            nn.Linear(512, 1),
            nn.Sigmoid()
        )
    
    def forward(self, xb):
        return self.network(xb)

In [91]:
def accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim=1)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))

  
@torch.no_grad()
def evaluate(model, val_loader):
    model.eval()
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)

  
def fit(epochs, lr, model, train_loader, val_loader, opt_func = torch.optim.SGD):
    
    history = []
    optimizer = opt_func(model.parameters(),lr)
    for epoch in range(epochs):
        
        model.train()
        train_losses = []
        for batch in train_loader:
            loss = model.training_step(batch)
            train_losses.append(loss)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            
        result = evaluate(model, val_loader)
        result['train_loss'] = torch.stack(train_losses).mean().item()
        model.epoch_end(epoch, result)
        history.append(result)
    
    return history

In [93]:
model = NaturalSceneClassification()

In [100]:
num_epochs = 30
opt_func = torch.optim.Adam
lr = 0.001
#fitting the model on training data and record the result after each epoch
history = fit(num_epochs, lr, model, train_dl, val_dl, opt_func)

RuntimeError: mat1 and mat2 shapes cannot be multiplied (30x10368 and 6272x512)

In [ ]:
%debug

> /usr/local/Anaconda/envs/py3.9/lib/python3.9/site-packages/torch/nn/modules/linear.py(103)forward()
    101 
    102     def forward(self, input: Tensor) -> Tensor:
--> 103         return F.linear(input, self.weight, self.bias)
    104 
    105     def extra_repr(self) -> str:

ipdb> whos
*** NameError: name 'whos' is not defined


In [97]:
model.training_step(batch)

NameError: name 'batch' is not defined

In [99]:
%debug

> /tmp/ipykernel_1927558/2448957765.py(1)<module>()
----> 1 model.training_step(batch)

ipdb> locals()


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



ipdb> locals()
ipdb> args
ipdb> args()
ipdb> locals
<built-in function locals>
ipdb> locals()


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



ipdb> images
*** NameError: name 'images' is not defined
ipdb> whos
*** NameError: name 'whos' is not defined
ipdb> s
